In [24]:
import os, json, time, datetime as dt, csv, pathlib
from typing import Dict, List
import requests
import pandas as pd
import bs4 as beautifulsoup
from dotenv import load_dotenv
import yfinance as yf

In [25]:
DATA_PATH = pathlib.Path('../data/raw')
DATA_PATH.mkdir(parents=True, exist_ok=True)
load_dotenv()
API_Key = os.getenv('ALPHAVANTAGE_API_KEY')
print("Your key is Imported :" , bool(API_Key))

Your key is Imported : True


In [26]:
def safe_stamp():
    return dt.datetime.now().strftime("%Y%m%d-%H%M%S")

def safe_filename(prefix: str, meta: Dict[str, str]) -> str:
    mid = "_".join([f"{k}-{str(v).replace(' ', '-')[:20]}" for k, v in meta.items()])
    return f"{prefix}_{mid}_{safe_stamp()}.csv"

def validate_df(df, required_cols, dtypes_map):
    msgs = {}
    missing = [c for c in required_cols if c not in df.columns]
    for col, dtype in dtypes_map.items():
        if col in df.columns:
            try:
                if dtype == 'datetime64[ns]':
                    pd.to_datetime(df[col])
                elif dtype == 'float':
                    pd.to_numeric(df[col])
            except Exception as e:
                msgs[f'dtype_{col}'] = f"Failed to coerce {col} to {dtype}: {e}"
    na_counts = df.isna().sum().sum()
    msgs['na_total'] = f"Total NA values: {na_counts}"
    return msgs